In [2]:
import requests
import json
import pandas as pd
from pprint import pprint

### Data Retrieval

In [25]:
data = pd.read_csv("../Test Data/Rail_Equipment_Accident_Incident_Data.csv")
data.shape

FileNotFoundError: [Errno 2] No such file or directory: '.../Test Data/Rail_Equipment_Accident_Incident_Data.csv'

In [23]:
from_year = data["Report Year"].min()
to_year = data["Report Year"].max()
print(f"The datasets records year from {from_year} to {to_year}")

The datasets records year from 1975.0 to 2022.0


## Fatality by state

#### Period" for last 11 years: 2010 - 2020

In [9]:
data_fatality = data[["State Name","Report Year","Total Persons Killed"]]
data_fatality = data_fatality.loc[(data_fatality["Report Year"] >= 2010.0) & (data_fatality["Report Year"]<=2020.0) ]
data_fatality.head()

,State Name,Report Year,Total Persons Killed
0,ALASKA,2011.0,0.0
1,ALABAMA,2010.0,0.0
5,ALABAMA,2010.0,0.0
6,ALABAMA,2010.0,0.0
8,ALABAMA,2013.0,0.0


In [14]:
state_group = data_fatality.groupby("State Name")
state_group["Total Persons Killed"].sum()

State Name
ALABAMA                 18.0
ALASKA                   0.0
ARIZONA                  8.0
ARKANSAS                11.0
CALIFORNIA              82.0
COLORADO                17.0
CONNECTICUT              0.0
DELAWARE                 0.0
DISTRICT OF COLUMBIA     0.0
FLORIDA                 70.0
GEORGIA                 12.0
IDAHO                    5.0
ILLINOIS                50.0
INDIANA                 21.0
IOWA                     8.0
KANSAS                   9.0
KENTUCKY                 5.0
LOUISIANA               21.0
MAINE                    2.0
MARYLAND                 7.0
MASSACHUSETTS            5.0
MICHIGAN                 8.0
MINNESOTA               12.0
MISSISSIPPI             24.0
MISSOURI                10.0
MONTANA                  9.0
NEBRASKA                 5.0
NEVADA                  12.0
NEW HAMPSHIRE            0.0
NEW JERSEY               7.0
NEW MEXICO               7.0
NEW YORK                23.0
NORTH CAROLINA          13.0
NORTH DAKOTA             8.0
OHI

### Question to be addressed
1. accidents/incidents for recent years (bar charts)
2. whether train becoming safer (line charts of injury rates, fatality rates)
3. which contribute to accidents:
check speed, temperature (bar charts)
4. Which region has highest vehicle rates (heatmap?

C:\Users\Eva\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (26,42,45,46,49,58,60,61,62,63,64,65,66,67,68,70,129,130,135) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(214157, 159)

In [ ]:
data.shape 